In [2]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Tell Dash to look for assets one level up
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)

app.title = "🚿 Bathroom Carousel"

# 📦 Sous-catégories dans assets/object/bathroom/
BATHROOM_ITEMS = sorted([
    item for item in os.listdir("../assets/object/bathroom")
    if os.path.isdir(os.path.join("../assets/object/bathroom", item))
])

# 📦 Dictionnaire {item: [liste d’images absolues]}
bathroom_images = {
    item: sorted([
        os.path.abspath(os.path.join("..", "assets", "object", "bathroom", item, img))
        for img in os.listdir(os.path.join("../assets/object/bathroom", item))
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    for item in BATHROOM_ITEMS
}

# 🖼️ Layout principal
app.layout = html.Div([
    html.H2("🚿 Bathroom Carousel", style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div([
        html.Label("Select bathroom item:"),
        dcc.Dropdown(
            id='item-dropdown',
            options=[{
                "label": html.Div(item.replace("_", " ").capitalize()),
                "value": item
            } for item in BATHROOM_ITEMS],
            value=BATHROOM_ITEMS[0] if BATHROOM_ITEMS else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Afficher le carousel
@app.callback(
    Output('carousel-container', 'children'),
    Input('item-dropdown', 'value')
)
def update_carousel(item):
    images = bathroom_images.get(item, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    # ✅ Get filename + relative path for Dash
    filename = os.path.basename(images[0])
    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))
    dash_path = f"/assets/{relative_path}"

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}),

            html.Img(id="carousel-image", src=dash_path,
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),

            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginLeft": "15px"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),

        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px"})
    ])

# 🔁 Navigation flèches
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))  # ✅ Convert abs → web path
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"


import threading
import time

def keep_dash_running():
    try:
        app.run(port=8804, debug=False, use_reloader=False)
    except:
        print("Dash server already running or port in use")

# 🚀 Lancer
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8804)


